In [102]:
# !pip install opensubtitlescom

In [106]:
# method to download and save imdb data to data/raw/
import requests

def download_data(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as file:
        file.write(response.content)

url = 'https://datasets.imdbws.com/'
basics = 'title.basics.tsv.gz'
ratings = 'title.ratings.tsv.gz'
path = '../data/raw/'

download_data(f'{url}{ratings}', f'{path}{ratings}')
download_data(f'{url}{basics}', f'{path}{basics}')

In [107]:
import pandas as pd

# load the data
basics_df = pd.read_csv(f'{path}{basics}', sep='\t', na_values='\\N')
ratings_df = pd.read_csv(f'{path}{ratings}', sep='\t', na_values='\\N')

C:\Users\kyle0\AppData\Local\Temp\ipykernel_33080\1510432930.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics_df = pd.read_csv(f'{path}{basics}', sep='\t', na_values='\\N')


In [108]:
# filter out only movies
movies_df = basics_df[basics_df['titleType'] == 'movie']

In [110]:
movies_df = movies_df.merge(ratings_df, on='tconst')

In [111]:
# sort by number of votes
movies_df = movies_df.sort_values('numVotes', ascending=False)

In [112]:
# remove adult movies
movies_df = movies_df[movies_df['isAdult'] == 0]

In [113]:
# make new column called imdb_id. This is the same as tconst but with 'tt' removed
movies_df['imdb_id'] = movies_df['tconst'].str[2:]

# now remove leading zeroes (for the subtitle api)
movies_df['imdb_id'] = movies_df['imdb_id'].str.lstrip('0')

In [118]:
df = movies_df[['tconst', 'imdb_id', 'primaryTitle', 'startYear', 'averageRating', 'numVotes']]
df = df.reset_index(drop=True)

# drop startYear if it is NaN
df = df.dropna(subset=['startYear'])

df['startYear'] = df['startYear'].astype(int)
df['averageRating'] = df['averageRating'].astype(float)
df['numVotes'] = df['numVotes'].astype(int)

# save as tsv
df[:20000].to_csv('../data/processed/movies.tsv', sep='\t', index=False)

In [98]:
from opensubtitlescom import OpenSubtitles
import json
import os
from dotenv import load_dotenv
load_dotenv()  # Loads variables from the .env file

APP_NAME = os.getenv('APP_NAME')
API_KEY = os.getenv('API_KEY')
MY_USERNAME = os.getenv('MY_USERNAME')
MY_PASSWORD = os.getenv('MY_PASSWORD')

# Initialize the OpenSubtitles client
subtitles = OpenSubtitles(APP_NAME, API_KEY)
subtitles.login(MY_USERNAME, MY_PASSWORD)

In [116]:
# list movie ids
ids = df['imdb_id'][:20000].tolist()
# ids

In [117]:
subtitle_path = "../data/raw/subtitles/"

for i in ids:
    # Search for subtitles based on imdb_id
    response = subtitles.search(imdb_id=i, languages="en", order_by="download_count", order_direction="desc")

    # Convert the response to a Json format
    response_json = response.to_json()
    r = json.loads(response_json)
    title = r['data'][0]['title'] # Get the title of the movie

    # Remove special characters but keep spaces
    title = ''.join(e for e in title if e.isalnum() or e.isspace())

    # Check if file exists
    if os.path.exists(f'{subtitle_path}{i} - {title}.txt'):
        continue

    # Download and parse the subtitle object
    srt = subtitles.download_and_parse(response.data[0])

    # Save
    with open(f'{subtitle_path}{i} - {title}.txt', 'w', encoding='utf-8') as f:
        for i in srt:
            # remove '<i>' and '</i>' tags
            i.content = i.content.replace('<i>', '').replace('</i>', '')
            f.write(i.content + '\n')


API `download` failed with 406: {"requests":100,"remaining":0,"message":"You have downloaded your allowed 100 subtitles for 24h.Your quota will be renewed in 07 hours and 21 minutes (2024-04-08 23:59:59 UTC) ts=1712594281 ","reset_time":"07 hours and 21 minutes","reset_time_utc":"2024-04-08T23:59:59.000Z","uk":"app_ud_94434","uid":680074,"ts":1712594281}


OpenSubtitlesException: Failed with HTTP 406 Client Error: Not Acceptable for url: https://api.opensubtitles.com/api/v1/download: 406 Client Error: Not Acceptable for url: https://api.opensubtitles.com/api/v1/download